In [3]:
import sys
import sympy
from sympy import *
from sympy.abc import x, n, z, t, k
from sympy.core.cache import *
    
clear_cache()    
    
init_printing(use_latex='mathjax') # for nice printing, a-la' TeX

sys.setrecursionlimit(100000)

In [176]:
from itertools import *

# Difference analysis of Polynomial sequences

Let $\{a_{n}\}_{n\in\mathbb{N}}$ be a sequence of Natural numbers. Assume that each coefficient $a_{n}$ can be written as a polynomial $\mathcal{A}_{k}$, of order $k$, in the variable $n$, namely: 
$$ a_{n} = \hat{a}_{0} + \hat{a}_{1}n + \ldots + \hat{a}_{k}n^{k} = \mathcal{A}_{k}(n)$$
where $\hat{a}_{i}\in\mathbb{Q}$ and $\hat{a}_{k}\neq0$.

In [88]:
def first_sequence_poly(n): return 1+n+2*n**2

In [104]:
first_sequence_exploded = [first_sequence_poly(i) for i in range(14)]
first_sequence_exploded

[1, 4, 11, 22, 37, 56, 79, 106, 137, 172, 211, 254, 301, 352]

In [248]:
def evaluate_poly_sym(polynomial):
    return lambda n: Poly(polynomial(n), polynomial.gen)

def difference(polynomial):
    return lambda n: (polynomial(n+1) - polynomial(n)).simplify()

def difs(sequence):
    
    if not sequence: return []
    elif len(sequence) is 1: return []
    else:
        n, m, *rest = sequence
        return [m-n] + difs([m]+rest)
        
def constant(l):
    if not l: return True
    else:
        car, *cdr = l
        return all(map(lambda a: car == a, cdr))
        
def difLists(lists):
    if not lists: return []
    else:
        l, *rest = lists
        if constant(l): return lists
        else: return difLists([difs(l)]+lists)
        
def firstDifs(sequence):
    return reversed(list(map(lambda l: l[0], difLists([sequence]))))

def genDifs(sequence):
    return reversed(list(map(lambda l: l[-1], difLists([sequence]))))

def nextD(lasts):
    new_coeff = list(accumulate(lasts))[-1]
    new_lasts = [new_coeff]
    for i in range(1,len(lasts)): new_lasts.append(new_lasts[i-1]-lasts[i-1])
    assert new_lasts[-1] == lasts[-1]
    return new_lasts

def next_element(sequence):
    return nextD(list(genDifs(sequence)))[0]
    
def continue_sequence(sequence):
    lasts = list(genDifs(sequence))
    while True:
        lasts = nextD(lasts)
        yield lasts[0]

In [138]:
f=first_sequence_poly(n+1)
f

             2    
n + 2⋅(n + 1)  + 2

In [126]:
differentiated_poly = difference(first_sequence_poly)(n) # keep it simbolically
mapped_difference = map(lambda i: differentiated_poly.subs(n,i), range(13))
list(mapped_difference)

[3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51]

In [127]:
difs(first_sequence_exploded)

[3, 7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51]

In [230]:
second_sequence = [-12, -11, 6, 45, 112, 213, 354, 541, 780, 1077]

In [231]:
first_difference_sequence = difs(second_sequence)
first_difference_sequence

[1, 17, 39, 67, 101, 141, 187, 239, 297]

In [232]:
second_difference_sequence = difs(first_difference_sequence)
second_difference_sequence

[16, 22, 28, 34, 40, 46, 52, 58]

In [233]:
third_difference_sequence = difs(second_difference_sequence)
third_difference_sequence

[6, 6, 6, 6, 6, 6, 6]

In [234]:
difLists([second_sequence])

[[6, 6, 6, 6, 6, 6, 6], [16, 22, 28, 34, 40, 46, 52, 58], [1, 17, 39, 67, 101,
 141, 187, 239, 297], [-12, -11, 6, 45, 112, 213, 354, 541, 780, 1077]]

In [235]:
list(firstDifs(second_sequence))

[-12, 1, 16, 6]

In [236]:
lasts = list(genDifs(second_sequence))
lasts

[1077, 297, 58, 6]

In [237]:
nextD(lasts)

[1438, 361, 64, 6]

In [238]:
next_element(second_sequence)

1438

In [239]:
take(continue_sequence(second_sequence),20)

[1438, 1869, 2376, 2965, 3642, 4413, 5284, 6261, 7350, 8557, 9888, 11349, 1294
6, 14685, 16572, 18613, 20814, 23181, 25720, 28437]

In [240]:
fibs = [fibonacci(i) for i in range(20)]
fibs

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584,
 4181]

In [241]:
take(continue_sequence(fibs),20)

[13530, 142065, 1357325, 9618830, 54023390, 254254950, 1041111056, 3806846406,
 12666486037, 38897569993, 111464146280, 300666911313, 768860983418, 187480503
2081, 4380558827899, 9848298629580, 21378856330309, 44949237172189, 9177525052
2398, 182390568524187]

In [249]:
difLists([fibs])

[[4181], [-2584, 1597], [1597, -987, 610], [-987, 610, -377, 233], [610, -377,
 233, -144, 89], [-377, 233, -144, 89, -55, 34], [233, -144, 89, -55, 34, -21,
 13], [-144, 89, -55, 34, -21, 13, -8, 5], [89, -55, 34, -21, 13, -8, 5, -3, 2
], [-55, 34, -21, 13, -8, 5, -3, 2, -1, 1], [34, -21, 13, -8, 5, -3, 2, -1, 1,
 0, 1], [-21, 13, -8, 5, -3, 2, -1, 1, 0, 1, 1, 2], [13, -8, 5, -3, 2, -1, 1, 
0, 1, 1, 2, 3, 5], [-8, 5, -3, 2, -1, 1, 0, 1, 1, 2, 3, 5, 8, 13], [5, -3, 2, 
-1, 1, 0, 1, 1, 2, 3, 5, 8, 13, 21, 34], [-3, 2, -1, 1, 0, 1, 1, 2, 3, 5, 8, 1
3, 21, 34, 55, 89], [2, -1, 1, 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 2
33], [-1, 1, 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610], [1,
 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597], [0, 
1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 418
1]]